In [7]:
from src.deep_learning.datahandling.dataset import DataLoader, CSVImageDataset
from src.deep_learning.optimizer.optimizer import Momentum
from src.deep_learning.RGrad.transform import ReLUBlock, Flatten, Transform
import src.deep_learning.RGrad.function
from src.deep_learning.RGrad.function import cross_entropy
from tqdm import tqdm

In [2]:
import numpy as np
import importlib

In [3]:
mnist_train_dataset = CSVImageDataset('/home/william/Documents/ML_Projects/AlgorithmsFFP/datasets/mnist_train.csv', (28, 28))
mnist_test_dataset = CSVImageDataset('/home/william/Documents/ML_Projects/AlgorithmsFFP/datasets/mnist_test.csv', (28, 28))

In [4]:
train_dataloader = DataLoader(mnist_train_dataset, 16, shuffle=True)
test_dataloader = DataLoader(mnist_test_dataset, 16, shuffle=False)

In [5]:
class MLP(Transform):

    def __init__(self):
        self.flatten = Flatten()
        self.relu1 = ReLUBlock(784, 60)
        self.relu2 = ReLUBlock(60, 60)
        self.relu3 = ReLUBlock(60, 10)
    
    def __call__(self, inpt):
        return self.relu3(self.relu2(self.relu1(self.flatten(inpt))))

In [9]:
model = MLP()
optimizer = Momentum(model.params(), 0.9, 0.01)

In [10]:
def accuracy(model, dataloader):
    num_right = 0
    num_wrong = 0
    for inpt, labels in dataloader:
        logits = model(inpt)
        predictions = np.argmax(logits.elems, axis=1)
        num_right_batch = np.sum(predictions==labels.elems)
        num_right += num_right_batch
        num_wrong += (len(predictions) - num_right_batch)
    return num_right, num_wrong

In [11]:
num_right, num_wrong = accuracy(model, test_dataloader)
print(f'acc: {num_right/(num_right + num_wrong)}')

acc: 0.1001


In [13]:
len(mnist_train_dataset.datapoints)//16

3750

In [15]:
losses = []
batch_num = 0
for inpt, labels in train_dataloader:
    if batch_num % 100 == 0:
        print(batch_num)
    logits = model(inpt)
    loss = cross_entropy(logits, labels)
    loss.backward()
    losses.append(loss.elems)
    optimizer.update()
    loss.zero_grads()
    batch_num += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700


In [34]:
np.random.uniform(0, 1, size=(2, 2))

array([[0.79225542, 0.92685544],
       [0.31656808, 0.21713027]])

In [16]:
num_right, num_wrong = accuracy(model, test_dataloader)
print(f'acc: {num_right/(num_right + num_wrong)}')

acc: 0.8518


In [17]:
num_right, num_wrong = accuracy(model, train_dataloader)
print(f'acc: {num_right/(num_right + num_wrong)}')

acc: 0.85025
